В этом уроке применим линейную регрессию на практике – попробуем предсказать стоимость машин и понять, 
от каких факторов зависит ценообразование на автомобили. Помимо этого узнаем, какие переменные важны для прогнозирования 
и насколько хорошо полученная модель описывает данные. 

Задание:
1. Загрузите данные, проверьте правильность, наличие пропущенных значений, типы данных.
2. Создайте новый признак – марку автомобиля (company). Машины каких производителей встречаются в датасете? Далее исправьте названия и проверьте изменения.
3. Оставьте только часть предикторов, после чего посчитайте корреляцию между price и другими переменными.
4. Преобразуйте категориальные переменные с помощью pd.get_dummies(). 
5. Постройте модель с одним предиктором цены – horsepower. Какой процент изменчивости объясняет полученная модель? (\(R^2\))
6. Далее – две модели (со всеми предикторами и со всеми, кроме марок машин). Обратите внимание на изменения в \(R^2\), коэффициентах и их значимости. Какую модель лучше оставить? 
7. Заполните пропуски в результатах.

In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.stats as ss
from scipy import stats
import pingouin as pg
from scipy.stats import levene
import scipy

In [5]:
cars = pd.read_csv('cars.csv')
cars

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,-1,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0
201,202,-1,volvo 144ea,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0
202,203,-1,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0
203,204,-1,volvo 246,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470.0


Загрузите данные, проверьте типы, пропущенные значения. 

Сколько пропущенных значений встретилось в датасете?

In [6]:
cars.isna().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


Сгенерируем первый признак.

Использовать полное название машины – не самый хороший вариант, поэтому создадим новый признак – марку автомобиля (company). 
Для этого используйте столбец CarName, разбейте значения ячеек по пробелу и запишите в колонку первый элемент. 

Например:
'audi 100 ls' → 'audi'

Сколько всего уникальных марок машины встречается в датасете? Столбец CarName с полным названием машины удалите из датасета, 
а также car_ID, они не пригодятся для дальнейшего анализа.

In [7]:
cars['company'] = cars.CarName.apply(lambda x: x.split(' ')[0])

In [8]:
cars

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,company
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,-1,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,...,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0,volvo
201,202,-1,volvo 144ea,gas,turbo,four,sedan,rwd,front,109.1,...,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0,volvo
202,203,-1,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,...,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0,volvo
203,204,-1,volvo 246,diesel,turbo,four,sedan,rwd,front,109.1,...,idi,3.01,3.40,23.0,106,4800,26,27,22470.0,volvo


In [9]:
cars.company.nunique()

28

In [10]:
cars = cars.drop(columns=['car_ID', 'CarName'])

In [11]:
cars

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,company
0,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,1,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,2,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,2,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,...,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0,volvo
201,-1,gas,turbo,four,sedan,rwd,front,109.1,188.8,68.8,...,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0,volvo
202,-1,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,...,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0,volvo
203,-1,diesel,turbo,four,sedan,rwd,front,109.1,188.8,68.9,...,idi,3.01,3.40,23.0,106,4800,26,27,22470.0,volvo


Теперь внимательнее посмотрите на уникальные значения company. Часть из них оказалась с ошибками!

- 'maxda' → 'mazda'
- 'Nissan' → 'nissan'
- 'porcshce' → 'porsche'
- 'toyouta' → 'toyota'
- 'vokswagen' & 'vw' → 'volkswagen'

Сколько уникальных производителей осталось в итоге?

In [12]:
cars['company'] = cars.company.str.lower().replace(regex={'maxda': 'mazda', 'porcshce': 'porsche', 'toyouta': 'toyota', 
                                             'vokswagen': 'volkswagen', 'vw': 'volkswagen'})

In [13]:
cars

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,company
0,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,1,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,2,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,2,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,...,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0,volvo
201,-1,gas,turbo,four,sedan,rwd,front,109.1,188.8,68.8,...,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0,volvo
202,-1,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,...,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0,volvo
203,-1,diesel,turbo,four,sedan,rwd,front,109.1,188.8,68.9,...,idi,3.01,3.40,23.0,106,4800,26,27,22470.0,volvo


In [14]:
cars.company.nunique()

22

Отлично! Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них:

'company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 
'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower'
также не забыв про то, что мы предсказываем – 'price'. 

После этого посчитайте корреляцию между price и другими переменными. Чему равна корреляция между price и horsepower? 
Ответ округлите до 2 знаков после точки.

Hint: можно использовать метод для датафрейма – .corr()

In [15]:
cars = cars[['company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 
      'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower', 'price']]
cars

,company,fueltype,aspiration,carbody,drivewheel,wheelbase,carlength,carwidth,curbweight,enginetype,cylindernumber,enginesize,boreratio,horsepower,price
0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111,13495.0
1,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111,16500.0
2,alfa-romero,gas,std,hatchback,rwd,94.5,171.2,65.5,2823,ohcv,six,152,2.68,154,16500.0
3,audi,gas,std,sedan,fwd,99.8,176.6,66.2,2337,ohc,four,109,3.19,102,13950.0
4,audi,gas,std,sedan,4wd,99.4,176.6,66.4,2824,ohc,five,136,3.19,115,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,2952,ohc,four,141,3.78,114,16845.0
201,volvo,gas,turbo,sedan,rwd,109.1,188.8,68.8,3049,ohc,four,141,3.78,160,19045.0
202,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,3012,ohcv,six,173,3.58,134,21485.0
203,volvo,diesel,turbo,sedan,rwd,109.1,188.8,68.9,3217,ohc,six,145,3.01,106,22470.0


In [16]:
cars_corr = cars.corr()
cars_corr

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price
wheelbase,1.000000,0.874587,0.795144,0.776386,0.569329,0.488750,0.353294,0.577816
carlength,0.874587,1.000000,0.841118,0.877728,0.683360,0.606454,0.552623,0.682920
carwidth,0.795144,0.841118,1.000000,0.867032,0.735433,0.559150,0.640732,0.759325
curbweight,0.776386,0.877728,0.867032,1.000000,0.850594,0.648480,0.750739,0.835305
enginesize,0.569329,0.683360,0.735433,0.850594,1.000000,0.583774,0.809769,0.874145
boreratio,0.488750,0.606454,0.559150,0.648480,0.583774,1.000000,0.573677,0.553173
horsepower,0.353294,0.552623,0.640732,0.750739,0.809769,0.573677,1.000000,0.808139
price,0.577816,0.682920,0.759325,0.835305,0.874145,0.553173,0.808139,1.000000


In [17]:
cars_corr.price.sort_values(ascending=False).round(2)

price         1.00
enginesize    0.87
curbweight    0.84
horsepower    0.81
carwidth      0.76
carlength     0.68
wheelbase     0.58
boreratio     0.55
Name: price, dtype: float64

Последний шаг в подготовке данных: линейная регрессия в python не справляется с категориальными переменными 
(типом object в pandas), поэтому давайте применим функцию под названием pd.get_dummies(). 
Она создаёт т.н. фиктивные переменные на основе изначальных категорий, представленные в виде 0 и 1. 
Официально такое преобразование называется One-Hot Encoding.

Для начала примените её только на категориальных переменных:

df_dummy = pd.get_dummies(data=cars[[список_столбцов_типа_object]], drop_first = True)

Сравните результат с изначальными данными. Что изменилось? Как теперь выглядят данные? 
Примечание: если вас интересует, что такое drop_first=True и зачем это нужно - загляните сюда.

Теперь примените её ко всему датафрейму (с тем же набором колонок, что и на предыдущем шаге). 
Не забудьте сохранить результат, он нам ещё понадобится :)

Сколько колонок теперь имеется в датасете? 

In [18]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   company         205 non-null    object 
 1   fueltype        205 non-null    object 
 2   aspiration      205 non-null    object 
 3   carbody         205 non-null    object 
 4   drivewheel      205 non-null    object 
 5   wheelbase       205 non-null    float64
 6   carlength       205 non-null    float64
 7   carwidth        205 non-null    float64
 8   curbweight      205 non-null    int64  
 9   enginetype      205 non-null    object 
 10  cylindernumber  205 non-null    object 
 11  enginesize      205 non-null    int64  
 12  boreratio       205 non-null    float64
 13  horsepower      205 non-null    int64  
 14  price           205 non-null    float64
dtypes: float64(5), int64(3), object(7)
memory usage: 24.1+ KB


In [19]:
df_dummy = pd.get_dummies(data=cars[['fueltype', 'aspiration', 'carbody', 'drivewheel',
                                    'enginetype', 'cylindernumber', 'company']], drop_first = True)
df_dummy.shape

(205, 41)

In [20]:
df_dummy

,fueltype_gas,aspiration_turbo,carbody_hardtop,carbody_hatchback,carbody_sedan,carbody_wagon,drivewheel_fwd,drivewheel_rwd,enginetype_dohcv,enginetype_l,...,company_nissan,company_peugeot,company_plymouth,company_porsche,company_renault,company_saab,company_subaru,company_toyota,company_volkswagen,company_volvo
0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
201,1,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
202,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
203,0,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [21]:
df_dummy = pd.get_dummies(data=cars,drop_first = True)
df_dummy.shape[1]

49

In [22]:
df_dummy

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price,company_audi,company_bmw,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,88.6,168.8,64.1,2548,130,3.47,111,13495.0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,88.6,168.8,64.1,2548,130,3.47,111,16500.0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,94.5,171.2,65.5,2823,152,2.68,154,16500.0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,99.8,176.6,66.2,2337,109,3.19,102,13950.0,1,0,...,1,0,0,0,0,1,0,0,0,0
4,99.4,176.6,66.4,2824,136,3.19,115,17450.0,1,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,109.1,188.8,68.9,2952,141,3.78,114,16845.0,0,0,...,1,0,0,0,0,1,0,0,0,0
201,109.1,188.8,68.8,3049,141,3.78,160,19045.0,0,0,...,1,0,0,0,0,1,0,0,0,0
202,109.1,188.8,68.9,3012,173,3.58,134,21485.0,0,0,...,0,0,1,0,0,0,1,0,0,0
203,109.1,188.8,68.9,3217,145,3.01,106,22470.0,0,0,...,1,0,0,0,0,0,1,0,0,0


Сначала построим небольшую модель всего с одним предиктором цены (price) – horsepower.

Какой процент изменчивости объясняет полученная модель? (округлите до целого)

In [23]:
import statsmodels.api as sm
import statsmodels.formula.api as smf 

In [24]:
results = smf.ols('price ~ horsepower', cars).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Tue, 31 Jan 2023   Prob (F-statistic):           1.48e-48
Time:                        23:00:10   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.0

In [25]:
# или

In [26]:
# Y = одномерный массив с ЗП, X - массив со всеми нужными нам НП
Y = cars.price
X = sm.add_constant(cars.horsepower)  # добавить константу, чтобы был свободный член
model = sm.OLS(Y, X)  # говорим модели, что у нас ЗП, а что НП
results = model.fit()  # строим регрессионную прямую
print(results.summary())  # смотрим результат

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Tue, 31 Jan 2023   Prob (F-statistic):           1.48e-48
Time:                        23:00:10   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3721.7615    929.849     -4.003      0.0

- **Где смотреть процент изменчивости?**
    
    В верхней правой части таблицы, пункт **R-squared**.

Ответ: 65%

Теперь построим ещё две модели:

модель со всеми предикторами
модель со всеми предикторами, кроме марок машин
Попробуем выбрать из этих моделей самую удачную! 
В каком-то смысле это искусство - при выборе моделей приходится ориентироваться не только на количественные показатели, 
но также на контекст её использования, опыт и здравый смысл.

Посмотрите на показатели всех трёх моделей. Выберите верные утверждения:

In [27]:
cars.columns

Index(['company', 'fueltype', 'aspiration', 'carbody', 'drivewheel',
       'wheelbase', 'carlength', 'carwidth', 'curbweight', 'enginetype',
       'cylindernumber', 'enginesize', 'boreratio', 'horsepower', 'price'],
      dtype='object')

In [28]:
results = smf.ols('price ~ company + fueltype + aspiration + carbody + drivewheel + wheelbase+carlength+carwidth+curbweight+enginetype+cylindernumber+enginesize+boreratio+horsepower', cars).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Tue, 31 Jan 2023   Prob (F-statistic):           4.86e-89
Time:                        23:00:10   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                -3.472e+04   1.17e+04     -2.959      0.004   -5.79e+04   -1.15e+04
company[T.audi]            437.1040   2125.971      0.206      0.837   -3761.885    4636.093
company[T.bmw]            7993.4998   2105.132      3.797      0.000    3835.671    1.22e+04
company[T.buick]          3431.0969   2472.252      1.388      0.167   -1451.829    8314.023
company[T.chevrolet]     -2107.0167   2054.269     -1.026      0.307   -6164.387    1950.353
company[T.dodge]         -3034.5530   1686.337     -1.799      0.074   -6365.224     296.118
company[T.honda]         -2328.7396   1622.141     -1.436      0.153   -5532.617     875.138
company[T.isuzu]          -926.1643   1819.489     -0.509      0.611   -4519.823    2667.495
company[T.jaguar]         2695.2664   2546.268      1.059      0.291   -2333.847    7724.379
company[T.mazda]         -1291.5657   1652.561     -0.782      0.436   -4555.527    1972.395
company[T.mercury]       -2762.0259   2786.701     -0.991      0.323   -8266.016    2741.965
company[T.mitsubishi]    -3699.6473   1650.046     -2.242      0.026   -6958.640    -440.654
company[T.nissan]        -1488.1217   1563.564     -0.952      0.343   -4576.305    1600.061
company[T.peugeot]       -2671.4595   1795.328     -1.488      0.139   -6217.398     874.479
company[T.plymouth]      -3052.1258   1670.829     -1.827      0.070   -6352.166     247.915
company[T.porsche]        4915.9715   2797.761      1.757      0.081    -609.864    1.04e+04
company[T.renault]       -2865.2197   2073.046     -1.382      0.169   -6959.676    1229.237
company[T.saab]            719.8222   1767.620      0.407      0.684   -2771.390    4211.035
company[T.subaru]        -1.164e+04   3416.221     -3.407      0.001   -1.84e+04   -4892.375
company[T.toyota]        -2608.7871   1539.320     -1.695      0.092   -5649.086     431.512
company[T.volkswagen]    -2261.9355   1606.886     -1.408      0.161   -5435.683     911.812
company[T.volvo]            68.2605   1943.170      0.035      0.972   -3769.679    3906.200
fueltype[T.gas]           -243.7792    908.310     -0.268      0.789   -2037.774    1550.216
aspiration[T.turbo]        596.6566    676.548      0.882      0.379    -739.588    1932.902
carbody[T.hardtop]       -2556.7157   1213.929     -2.106      0.037   -4954.337    -159.094
carbody[T.hatchback]     -3044.4196   1107.064     -2.750      0.007   -5230.972    -857.867
carbody[T.sedan]         -2256.9173   1154.971     -1.954      0.052   -4538.090      24.256
carbody[T.wagon]         -2815.4970   1223.156     -2.302      0.023   -5231.343    -399.651
drivewheel[T.fwd]         -169.4926    888.585     -0.191      0.849   -1924.531    1585.545
drivewheel[T.rwd]          231.9066   1039.685      0.223      0.824   -1821.566    2285.380
enginetype[T.d

In [29]:
results = smf.ols('price ~ fueltype + aspiration + carbody + drivewheel + wheelbase+carlength+carwidth+curbweight+enginetype+cylindernumber+enginesize+boreratio+horsepower', cars).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Tue, 31 Jan 2023   Prob (F-statistic):           9.86e-81
Time:                        23:00:11   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                  -1.7e+04    1.3e+04     -1.309      0.192   -4.26e+04    8625.219
fueltype[T.gas]          -2423.0935    975.579     -2.484      0.014   -4348.283    -497.904
aspiration[T.turbo]      -1139.2334    787.230     -1.447      0.150   -2692.738     414.271
carbody[T.hardtop]       -3728.0949   1519.366     -2.454      0.015   -6726.383    -729.807
carbody[T.hatchback]     -4639.1279   1294.594     -3.583      0.000   -7193.855   -2084.401
carbody[T.sedan]         -3337.7854   1356.075     -2.461      0.015   -6013.838    -661.732
carbody[T.wagon]         -4315.4661   1465.089     -2.946      0.004   -7206.646   -1424.287
drivewheel[T.fwd]          297.2402   1121.766      0.265      0.791   -1916.431    2510.912
drivewheel[T.rwd]         1939.1594   1226.536      1.581      0.116    -481.263    4359.582
enginetype[T.dohcv]      -1.713e+04   4540.120     -3.773      0.000   -2.61e+04   -8171.142
enginetype[T.l]           2176.0521   1332.818      1.633      0.104    -454.106    4806.211
enginetype[T.ohc]         4006.5687    897.437      4.464      0.000    2235.584    5777.553
enginetype[T.ohcf]        5421.2822   1388.078      3.906      0.000    2682.076    8160.488
enginetype[T.ohcv]       -5001.2242   1345.435     -3.717      0.000   -7656.280   -2346.169
enginetype[T.rotor]      -5196.6413   2084.532     -2.493      0.014   -9310.217   -1083.066
cylindernumber[T.five]   -1.389e+04   2891.599     -4.802      0.000   -1.96e+04   -8180.353
cylindernumber[T.four]   -1.618e+04   3243.610     -4.989      0.000   -2.26e+04   -9780.800
cylindernumber[T.six]    -1.175e+04   2287.516     -5.135      0.000   -1.63e+04   -7231.547
cylindernumber[T.three]  -9264.0553   4750.899     -1.950      0.053   -1.86e+04     111.279
cylindernumber[T.twelve] -1.214e+04   3386.507     -3.585      0.000   -1.88e+04   -5458.378
cylindernumber[T.two]    -5196.6413   2084.532     -2.493      0.014   -9310.217   -1083.066
wheelbase                   71.1868     87.028      0.818      0.414    -100.552     242.925
carlength                  -51.3497     50.341     -1.020      0.309    -150.692      47.993
carwidth                   541.8700    253.327      2.139      0.034      41.958    1041.782
curbweight                   2.9577      1.796      1.647      0.101      -0.585       6.501
enginesize                  36.0515     22.376      1.611      0.109      -8.105      80.208
boreratio                -2230.4519   1731.681     -1.288      0.199   -5647.719    1186.815
horsepower                  86.8164     16.717      5.193      0.000      53.827     119.806
==============================================================================
Omnibus:                       18.493   Durbin-Watson:                   1.249
Prob(Omnibus):                  0.000   Ja

Ответ: 
Большинство коэффициентов, связанных с марками машин, статистически незначимы
Если судить чисто по диагностическим показателям (вроде R2), то модель со всеми предикторами лучшая

Вы могли обратить внимание, что хотя марки машин и объясняют какую-то часть общей дисперсии в данных, 
эта часть не самая большая - около 4%. На фоне того, как эта переменная усложняет модель дополнительными статнезначимыми 
коэффициентами, мы можем принять решение выкинуть её из модели либо дополнительно переделать. 
Однозначно правильного решения тут нет.

Выбранная модель объясняет примерно 90% дисперсии (окр. до целого). Среди предикторов 10 из 27 оказались не значимыми (p > 0.05).
Пример интерпретации: при единичном изменении показателя horsepower, цена 
ВОЗРАСТАЕТ на 86.8164 (без округления).